In [10]:
import torch
import tarfile
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base


In [2]:
print(torch.__version__)

1.7.1


In [25]:
from torchvision.models import imagenet_classes


ImportError: cannot import name 'imagenet_classes'

In [4]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /home/ec2-user/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
input_shape = [1, 3, 224, 224]
trace = torch.jit.trace(model.float().eval(), torch.zeros(input_shape).float())
trace.save("model.pth")
with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("model.pth")

In [12]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("TorchVision-ResNet18-Neo")
prefix = compilation_job_name + "/model"
model_path = sess.upload_data(path="model.tar.gz", key_prefix=prefix)

data_shape = '{"input0":[1,3,224,224]}'
target_platform = {'Os': 'LINUX','Arch': 'X86_64'}
framework = "PYTORCH"
framework_version = "1.6"
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)


In [17]:
# Create a SageMaker client so you can submit a compilation job
sagemaker_client = boto3.client('sagemaker', region_name='us-west-2')

# Give your compilation job a name
compilation_job_name = 'getting-started-demo'
print(f'Compilation job for {compilation_job_name} started')

response = sagemaker_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': model_path,
        'DataInputConfig': data_shape,
        'Framework': framework.upper()
    },
    OutputConfig={
        'S3OutputLocation': compiled_model_path,
        'TargetPlatform': target_platform 
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900
    }
)
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

Compilation job for getting-started-demo started


NameError: name 's3_input_location' is not defined

In [77]:
s3_client = boto3.client('s3')
object_path = '{}/output/model-{}_{}.tar.gz'.format(compilation_job_name, target_platform['Os'], target_platform['Arch'])
neo_compiled_model = 'compiled-model.tar.gz'
s3_client.download_file(bucket, object_path, neo_compiled_model)


In [78]:
!mkdir model
!tar zfxv compiled-model.tar.gz -C model/
!zip compiled-model.zip model/*
s3_client.upload_file('compiled-model.zip', bucket, '{}/output/model-{}_{}.zip'.format(compilation_job_name, target_platform['Os'], target_platform['Arch']))



dlr.h
compiled_model.json
compiled.meta
libdlr.so
compiled.so
compiled.params
manifest
  adding: model/compiled.meta (deflated 66%)
  adding: model/compiled_model.json (deflated 93%)
  adding: model/compiled.params (deflated 7%)
  adding: model/compiled.so (deflated 78%)
  adding: model/dlr.h (deflated 83%)
  adding: model/libdlr.so (deflated 60%)
  adding: model/manifest (deflated 45%)
